In [1]:
import geopandas as gpd
import os
import warnings
import ast

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
rrfb_path = os.path.join(base_dir, "rrfb.shp")
unassociated_rrfb_path = os.path.join(base_dir, "unassociated_rrfb.shp")

# Load shapefiles
roads = gpd.read_file(roads_path)
crossings = gpd.read_file(crossings_path)
rrfb = gpd.read_file(rrfb_path)

# Convert CRS_LEG from string to list
crossings['CRS_LEG'] = crossings['CRS_LEG'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Reproject RRFBs to match roads CRS
rrfb = rrfb.to_crs(roads.crs)

# Add 'ID' column to RRFBs and assign sequential values
rrfb['ID'] = range(1, len(rrfb) + 1)

# Buffer RRFBs by 5 meters
rrfb['geometry'] = rrfb.geometry.buffer(1)

# Identify road segments that intersect with RRFBs
rrfb_road_segments = roads[roads.geometry.intersects(rrfb.unary_union)]
rrfb_road_ids = set(rrfb_road_segments['unique_id'])

# Create a dictionary to store the nearest junction for each RRFB
junctions = crossings[['JUNC_ID', 'geometry']].drop_duplicates()  # Get unique junction points
rrfb_to_junction = {}

for _, rrfb_row in rrfb.iterrows():
    rrfb_geom = rrfb_row.geometry
    distances = junctions.geometry.distance(rrfb_geom)
    nearest_junction_idx = distances.idxmin()
    nearest_junction_id = junctions.loc[nearest_junction_idx, 'JUNC_ID']
    rrfb_to_junction[rrfb_row['ID']] = nearest_junction_id

# Assign RRFB to crossings based on the nearest intersecting road segment
crossings['RRFB'] = 0  # Default to 0 (no RRFB)
for idx, row in crossings.iterrows():
    crs_leg = row['CRS_LEG']

    if isinstance(crs_leg, list):
        has_rrfb = any(seg in rrfb_road_ids for seg in crs_leg)

        # Ensure that the crossing is at the nearest junction for the RRFB
        if has_rrfb and row['JUNC_ID'] in rrfb_to_junction.values():
            crossings.at[idx, 'RRFB'] = 1  # Assign 1 for crossings with RRFB

# Identify unassociated RRFBs (those not linked to any road segment)
unassociated_rrfb = rrfb[~rrfb.intersects(rrfb_road_segments.unary_union)]

# Save unassociated RRFBs to a new shapefile
unassociated_rrfb.to_file(unassociated_rrfb_path, driver='ESRI Shapefile')

# Save the updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

# Output summary
print("Assigned RRFB to relevant crossings.")
print(f"Total crossings with RRFB: {crossings['RRFB'].sum()}")
print(f"There are {len(unassociated_rrfb)} unassociated RRFBs.")

Assigned RRFB to relevant crossings.
Total crossings with RRFB: 14
There are 0 unassociated RRFBs.
